# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
CONFIG =  or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 100

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
#  'Random': or_suite.agents.rl.random.randomAgent(),
#  'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
 'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
 'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
 'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[1.952 1.973 2.037 2.016 2.073 1.949 2.002 2.005 2.052 2.05 ]
 [2.947 3.043 2.995 2.965 3.02  2.994 3.003 3.001 3.036 3.001]
 [3.945 3.992 4.033 3.934 4.003 4.048 3.993 4.041 4.033 3.931]] [[0.937696 1.044271 0.983631 1.017744 1.047671 0.932399 1.015996 1.014975
  1.125296 1.0335  ]
 [2.008191 1.979151 2.058975 2.031775 2.1296   1.977964 2.064991 2.144999
  2.058704 1.972999]
 [3.063975 3.049936 2.951911 2.989644 2.946991 2.985696 3.104951 2.959319
  3.035911 2.730239]]
Mean and variance endomwnets:
[[1.973 2.026 1.999 1.942 2.007 2.019 1.98  2.024 2.078 2.011]
 [3.122 3.054 2.903 3.008 2.986 3.042 2.995 3.073 2.989 2.912]
 [4.01  4.024 3.901 3.92  3.968 4.035 4.051 3.966 3.963 4.027]] [[1.116271 1.065324 0.970999 0.950636 0.944951 0.980639 0.9916   0.983424
  0.987916 1.034879]
 [2.043116 1.959084 2.047591 1.899936 2.009804 2.082236 1.940975 2.019671
  1.986879 1.954256]
 [2.9119   3.143424 3.051199 3.1016   2.688976 3.171775 3.136399 2.930844
  3.093631

# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = {'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG), \
                      'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG), \
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG), \
                      'Budget' : lambda traj : or_suite.utils.times_out_of_budget(traj, CONFIG)}
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)}
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

FixedThreshold
Lower Solutions:
[[0.      0.13786]
 [0.      0.13786]
 [0.15706 0.     ]]
Writing to file data.csv
Guardrail-0.5
Lower and Upper Solutions:
[[0.      0.2062 ]
 [0.      0.2062 ]
 [0.23741 0.     ]]
[[0.      0.30157]
 [0.      0.30157]
 [0.34721 0.     ]]
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Writing to file data.csv
Guardrail-0.3
Lower and Upper Solutions:
[[0.      0.20699]
 [0.      0.20699]
 [0.23837 0.     ]]
[[0.      0.41497]
 [0.      0.41497]
 [0.47787 0.     ]]
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of Budget!
Out of

In [6]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [7]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)